In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("./BSSID.csv", sep=";")
df

,SSID,BSSID,Signal(%),X,Y
0,eduroam,f4:ea:b5:b0:d4:25,90,3.04,6.37
1,eduroam,f4:ea:b5:dc:15:25,65,4.27,6.43
2,eduroam,f4:ea:b5:dc:6d:96,100,5.65,6.50
3,eduroam,f4:ea:b5:dc:15:25,85,5.77,8.03
4,THGA,f4:ea:b5:dc:2f:15,70,7.38,2.16


In [2]:
from src.detect_user import detect_user_position, distance
from main import load_acces_points

position détecté
x : 5.581105709009553, y : 5.20220400119403


In [3]:
A, W = load_acces_points("./BSSID.csv")

In [4]:
M, objs = detect_user_position(A, W, d=distance)

In [5]:
print("[position détectée]  ")
print(f" M ({M[0]}, {M[1]})")

[position détectée]  
 M (5.581107323451348, 5.202204957113143)
